In [2]:
from generator import Task, LinearModel
from tokenizer import FloatTokenizer
from dataset import LinearModelDataset
from config import getConfig
from train import Trainer
from tqdm import tqdm
import torch
import math

In [3]:
cfg = getConfig()
trainer = Trainer(cfg)

Device: cpu


In [4]:
i = trainer.tokenizer.tokenToIdx("E1")
t = trainer.tokenizer.idxToToken(i)
c = trainer.tokenizer.idxToCategory(i)
print(i, t, c)

10014 E1 exponent


In [6]:
mask = trainer.tokenizer.conditionalMask(i)
mask[1:10] == False

exponent


tensor([ True, False,  True,  True, False, False,  True,  True,  True])

In [ ]:
def runBatch2(batch: dict) -> float:
        self.model.train()
        encoder_input = batch["encoder_input"].to(self.device) # (batch_size, seq_len)
        encoder_mask = batch["encoder_mask"].to(self.device) # (batch_size, 1, 1, seq_len)
        decoder_input = batch["decoder_input"].to(self.device) # (batch_size, seq_len)
        decoder_mask = batch["decoder_mask"].to(self.device) # (batch_size, 1, seq_len, seq_len)

        # forward pass
        encoder_output = self.model.encode(encoder_input, encoder_mask) # (batch_size, seq_len, d_model)
        decoder_output = self.model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) # (batch_size, seq_len, d_model)
        proj_output = self.model.projection(decoder_output) # (batch_size, seq_len, vocab_tgt_len)
        return proj_output

trainer.runBatch2 = runBatch2

In [ ]:
self = trainer
epoch = 1
train_dl, val_dl = self.getDataset(config=self.config, seed=2)
batch_iterator = tqdm(train_dl, desc=f"Epoch {epoch:02d}")

for batch in batch_iterator:
    proj_output = self.runBatch2(batch)
    break


In [ ]:
pred_idx = torch.argmax(proj_output, dim=-1)

In [ ]:
pred = torch.tensor(trainer.tokenizer.batchDecode(pred_idx))
print(pred)

In [ ]:
mse_loss = torch.nn.MSELoss(reduction='none')
tgt = batch["tgt"]

In [ ]:
loss = mse_loss(pred, tgt)
mask = ~torch.isnan(pred) & ~torch.isnan(tgt)
filtered_loss = loss[mask]
final_loss = filtered_loss.mean() if filtered_loss.numel() > 0 else torch.tensor(0.0)
print(final_loss)